# gtex v8 gwas sugroup analysis
## approximate bayes factor estimation to test colocalization between gwas and eqtl results 
# YoSon Park
## July 14, 2018

this notebook documents steps taken to run coloc on 110 traits for the GTEx v8 GWAS subgroup analysis/paper

* gwas summary statistics were obtained from various public databases and processed by alvaro (im lab). 
* all genomic positions are mapped to hg38 to match gtex v8 eqtl results and most missing variants were imputed using impg. 
* gtex v8 eqtl analysis results were obtained from dbGaP

In [1]:
# import some of the basic python modules for data formatting, etc
import os, sys
import pandas
import numpy

before the full scale analyses, these 10 traits are first downloaded and tested:
```
Public GWAS:

1. Coronary artery disease (CAD consortium):
imputed_CARDIoGRAM_C4D_CAD_ADDITIVE.txt.gz
2. Tobacco: ever smoker vs. never smoker (TAG consortium):
imputed_tag.evrsmk.tbl.txt.gz
3. Height (GIANT consortium):
imputed_GIANT_HEIGHT.txt.gz
5. Inflammatory Bowel Disease (IBD consortium):
imputed_IBD.EUR.Inflammatory_Bowel_Disease.txt.gz
6. Hypertension (diastolic, systolic)

UK Biobank:
1. asthma: imputed_UKB_20002_1111_self_reported_asthma.txt.gz
2. migraine:  imputed_UKB_20002_1265_self_reported_migraine.txt.gz
3. irritable bowel syndrome:  imputed_UKB_20002_1154_self_reported_irritable_bowel_syndrome.txt.gz
4. DVT:  imputed_UKB_20002_1094_self_reported_deep_venous_thrombosis_dvt.txt.gz
5. osteoporosis:  imputed_UKB_20002_1309_self_reported_osteoporosis.txt.gz
6. BMI: imputed_UKB_21001_Body_mass_index_BMI.txt.gz
```

In [ ]:
%%bash

# Public:
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_CARDIoGRAM_C4D_CAD_ADDITIVE.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_tag.evrsmk.tbl.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_GIANT_HEIGHT.txt.gz
# wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_SSGAC_Education_Years_Pooled.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_IBD.EUR.Inflammatory_Bowel_Disease.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_ICBP_DiastolicPressure.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_ICBP_SystolicPressure.txt.gz
    
# UK Biobank:
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_UKB_20002_1111_self_reported_asthma.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_UKB_20002_1265_self_reported_migraine.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_UKB_20002_1154_self_reported_irritable_bowel_syndrome.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_UKB_20002_1094_self_reported_deep_venous_thrombosis_dvt.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_UKB_20002_1309_self_reported_osteoporosis.txt.gz
wget https://storage.googleapis.com/summary_stats_imputation/processed_summary_imputation/imputed_UKB_21001_Body_mass_index_BMI.txt.gz

and these were agreed-upon primary tissues for analyses:
```
Muscle - Skeletal
Skin - Sun Exposed (Lower leg)
Thyroid
Adipose - Subcutaneous
Artery - Tibial
Lung
Nerve - Tibial
Esophagus - Mucosa
Whole Blood
Heart - Left Ventricle 

```

'egenes' were defined by per-tissue q-value < 0.05

In [ ]:
%%bash

#!/bin/bash

mkdir -p GTEx_Analysis_v8_eQTL_egene_list
cd GTEx_Analysis_v8_eQTL

for file in *egenes*
do

# head -2
# gene_id gene_name       gene_chr        gene_start      gene_end        strand  num_var beta_shape1     beta_shape2     true_df pval_true_df    variant_id      tss_distance    chr     variant_pos     ref     alt     num_alt_per_site        rs_id_dbSNP150_GRCh38p7 minor_allele_samples    minor_allele_count      maf     ref_factor      pval_nominal    slope   slope_se        pval_perm       pval_beta       qval    pval_nominal_threshold
# ENSG00000227232.5       unprocessed_pseudogene  chr1    14410   29553   -       1361    1.03263 304.688 532.69  1.77781e-12     chr1_666028_G_A_b38     636475  chr1    666028  G       A       1       rs143332664     115     117     0.0948136       1       7.23705e-14     0.552584        0.072093        9.999e-05       2.66233e-10     3.23444e-10     0.000209918

  tissue=$(echo ${file} | sed 's/.v8.egenes.txt.gz//')
  gzip -dc ${file} | awk '($29<0.05 && $1!~"gene_id"){print $1}' | sort -u > ../GTEx_Analysis_v8_eQTL_egene_list/${tissue}.egenes.lst

done


In [2]:
%%bash

wc -l GTEx_Analysis_v8_eQTL_egene_list/*lst

   15578 GTEx_Analysis_v8_eQTL_egene_list/Adipose_Subcutaneous.egenes.lst
   12485 GTEx_Analysis_v8_eQTL_egene_list/Adipose_Visceral_Omentum.egenes.lst
    8187 GTEx_Analysis_v8_eQTL_egene_list/Adrenal_Gland.egenes.lst
   12492 GTEx_Analysis_v8_eQTL_egene_list/Artery_Aorta.egenes.lst
    6352 GTEx_Analysis_v8_eQTL_egene_list/Artery_Coronary.egenes.lst
   15026 GTEx_Analysis_v8_eQTL_egene_list/Artery_Tibial.egenes.lst
    3730 GTEx_Analysis_v8_eQTL_egene_list/Brain_Amygdala.egenes.lst
    5636 GTEx_Analysis_v8_eQTL_egene_list/Brain_Anterior_cingulate_cortex_BA24.egenes.lst
    8395 GTEx_Analysis_v8_eQTL_egene_list/Brain_Caudate_basal_ganglia.egenes.lst
   10039 GTEx_Analysis_v8_eQTL_egene_list/Brain_Cerebellar_Hemisphere.egenes.lst
   11235 GTEx_Analysis_v8_eQTL_egene_list/Brain_Cerebellum.egenes.lst
    9078 GTEx_Analysis_v8_eQTL_egene_list/Brain_Cortex.egenes.lst
    7371 GTEx_Analysis_v8_eQTL_egene_list/Brain_Frontal_Cortex_BA9.egenes.lst
    5525 GTEx_Analysis_v8_eQTL_egene_list/Bra

for current analyses, priors are estimated using enloc. p1, p2 and p12 priors are in the file `coloc_prior_est_by_enloc.txt` generated and distributed by William.

In [2]:
priors = pandas.read_csv('coloc_prior_est_by_enloc.txt', sep='\t', header=0)
priors.head()

,trait,tissue,p1,p2,p12
0,CAD,Adipose_Subcutaneous,0.000021,0.002597,0.000009
1,CAD,Adipose_Visceral_Omentum,0.000023,0.002071,0.000007
2,CAD,Adrenal_Gland,0.000025,0.001276,0.000004
3,CAD,Artery_Aorta,0.000018,0.001988,0.000012
4,CAD,Artery_Coronary,0.000024,0.001085,0.000005


In [3]:
priors.trait.unique()

array(['CAD', 'Height', 'Hypertension', 'IBD', 'TAG', 'UKB_asthma',
       'UKB_BMI', 'UKB_DVT', 'UKB_IBS', 'UKB_migraine',
       'UKB_osteoporosis'], dtype=object)

for consistence, adding file-name-based trait values

In [4]:
# making priors file 'trait' as 'trait0'
priors['trait0'] = priors['trait']
# changing 'trait' to refer to file names
# not sure which dataset Hypertension refers to. leaving it for now.
priors.replace({'trait' : { 'CAD' : 'CARDIoGRAM_C4D_CAD_ADDITIVE', 
                           'Height' : 'GIANT_HEIGHT', 
                           'IBD' : 'IBD.EUR.Inflammatory_Bowel_Disease', 
                           'TAG' : 'tag.evrsmk.tbl', 
                           'UKB_BMI' : 'UKB_21001_Body_mass_index_BMI', 
                           'UKB_DVT' : 'UKB_20002_1094_self_reported_deep_venous_thrombosis_dvt', 
                           'UKB_IBS' : 'UKB_20002_1154_self_reported_irritable_bowel_syndrome', 
                           'UKB_asthma' : 'UKB_20002_1111_self_reported_asthma',
                           'UKB_migraine' : 'UKB_20002_1265_self_reported_migraine',
                           'UKB_osteoporosis' : 'UKB_20002_1309_self_reported_osteoporosis'
                          }}, inplace=True)
priors.trait.unique()

array(['CARDIoGRAM_C4D_CAD_ADDITIVE', 'GIANT_HEIGHT', 'Hypertension',
       'IBD.EUR.Inflammatory_Bowel_Disease', 'tag.evrsmk.tbl',
       'UKB_20002_1111_self_reported_asthma',
       'UKB_21001_Body_mass_index_BMI',
       'UKB_20002_1094_self_reported_deep_venous_thrombosis_dvt',
       'UKB_20002_1154_self_reported_irritable_bowel_syndrome',
       'UKB_20002_1265_self_reported_migraine',
       'UKB_20002_1309_self_reported_osteoporosis'], dtype=object)

drop hypertension from current analysis due to ambiguity

In [5]:
priors = priors.loc[priors['trait'] != 'Hypertension']

saving temp priors file for coloc analyses with new trait names

In [8]:
priors.to_csv('coloc_prior_est_by_enloc_with_traitfile.txt', sep='\t', header=True, index=False)

gtex v8 sample sizes are assessed from count of samples included in the covariates file

In [ ]:
%%bash
for f in GTEx_Analysis_v8_eQTL_covariates/*.v8.covariates.txt; do 
  tissue=$(echo ${f} | sed 's:GTEx_Analysis_v8_eQTL_covariates/::g; s:.v8.covariates.txt::g')
  count=$(head -1 ${f} | awk '{print NF}')
  echo $tissue `expr $count - 1`
done >> gtex_v8_sample_count.txt

In [4]:
!head gtex_v8_sample_count.txt

Adipose_Subcutaneous 581
Adipose_Visceral_Omentum 469
Adrenal_Gland 233
Artery_Aorta 387
Artery_Coronary 213
Artery_Tibial 584
Brain_Amygdala 129
Brain_Anterior_cingulate_cortex_BA24 147
Brain_Caudate_basal_ganglia 194
Brain_Cerebellar_Hemisphere 175


example coloc files are made from allpairs eqtl results files and imputed gwas summary statistics

In [ ]:
#!/usr/bin/env python3

# making merged input per gene for colocalization analyses using gtex eqtl and imputed gwas results summary statistics 
# YoSon Park

# example arguements
# gwasfile = 'imputed_CARDIoGRAM_C4D_CAD_ADDITIVE.txt.gz'
# tissue = 'Whole_Blood'
# example run
# > python3 gtex_gwas_crossinput.py imputed_CARDIoGRAM_C4D_CAD_ADDITIVE.txt.gz Whole_Blood

import os, sys
import pandas

rootdir = '/work/gtex/'
eqtldir = rootdir + 'results/GTEx_Analysis_v8_eQTL/'
outdir = rootdir + 'coloc_v8/input_per_gene/'

samplesfile = rootdir + 'coloc_v8/gtex_v8_sample_count.txt'
samples = pandas.read_csv(samplesfile, sep='\t', header=None, names=['tissue', 'eqtl_n'])

gwasfile = sys.argv[1]
tissue = sys.argv[2]


# gwas file head -2 imputed_CARDIoGRAM_C4D_CAD_ADDITIVE.txt.gz
# variant_id	gtex_variant_id	chromosome	position	effect_allele	non_effect_allele	current_build	frequency	sample_size	zscore	pvalue	effect_size	standard_error	imputation_status	n_cases
# rs372260684	chr10_18924_A_C_b38	chr10	18924	C	A	hg38	0.08008658008658008	184305	1.2056708779464282	0.22794441814657265	NA	NA	imputed	60801

gwasdir = rootdir + 'coloc_v8/gwas/'
gwasheader = ['gwas_variant_id', 'eqtl_variant_id', 'chromosome', 'position', 'effect_allele', 'non_effect_allele', 'current_build', 'frequency', 'gwas_sample_size', 'gwas_zscore', 'gwas_pvalue', 'gwas_effect_size', 'gwas_standard_error', 'gwas_imputation_status', 'gwas_n_cases']
gwas = pandas.read_csv(gwasdir + gwasfile, sep='\t', header=0, names=gwasheader)
# for a lack of better convention, keeping file names as 'trait' names
trait = gwasfile.replace('imputed_', '')
trait = trait.replace('.txt.gz', '')
gwas['trait'] = trait

# example egene file 
egenesfile = eqtldir + 'GTEx_Analysis_v8_eQTL_egene_list/' + tissue + '.egenes.lst'
egenes = pandas.read_csv(egenesfile, sep='\t', header=None, names=['gene_id'])

# head -2 of eqtldir + 'GTEx_Analysis_v8_eQTL_all_associations_per_gene/'+ tissue + '/' tissue + '_ENSG00000110002.15.allpairs.txt'
# gene_id	variant_id	tss_distance	ma_samples	ma_count	maf	pval_nominal	slope	slope_se
# ENSG00000110002.15	chr11_123115454_T_C_b38	-999908	75	79	0.0589552	0.102627	0.080717	0.0493759

# samples are counted from the header of per-tissue eqtl covariate files
sample = samples.loc[samples['tissue']==tissue]['eqtl_n'].values[0]

# adding prefix 'eqtl' to distinguish from 'gwas' columns in the final merged input file
eqtlheader = ['gene_id', 'eqtl_variant_id', 'tss_distance', 'eqtl_ma_samples', 'eqtl_ma_count', 'eqtl_maf', 'eqtl_pval_nominal', 'eqtl_slope', 'eqtl_slope_se']

# iterate through egenes list
for gene in egenes.gene_id:
  allpairsfile = eqtldir + 'GTEx_Analysis_v8_eQTL_all_associations_per_gene/'+ tissue + '/' + tissue + '_' + gene + '.allpairs.txt'
  if os.path.isfile(allpairsfile):
    # reformatting file names for consistency
    outfile = outdir + tissue + '_w_' + trait + '_for_' + gene + '_coloc_input_file.txt'
    if not os.path.isfile(outfile):
      allpairs = pandas.read_csv(allpairsfile, sep='\t', header=0, names=eqtlheader)
      allpairs['tissue'] = tissue
      allpairs['eqtl_sample_size'] = sample
      merged = allpairs.merge(gwas, on='eqtl_variant_id', left_index=False, right_index=False, how='inner')
      if not merged.empty:
        merged.to_csv(outfile, sep='\t', header=True, index=False, na_rep='NA')
      else:
        print(gene, ' is not available for colocalization analysis')



resulting example coloc input file

In [6]:
!head -2 input_per_gene/Whole_Blood_w_CARDIoGRAM_C4D_CAD_ADDITIVE_for_ENSG00000000457.13_coloc_input_file.txt

gene_id	eqtl_variant_id	tss_distance	eqtl_ma_samples	eqtl_ma_count	eqtl_maf	eqtl_pval_nominal	eqtl_slope	eqtl_slope_se	tissue	eqtl_sample_size	gwas_variant_id	chromosome	position	eqtl_effect_allele	eqtl_non_effect_allele	current_build	frequency	gwas_sample_size	gwas_zscore	gwas_pvalue	gwas_effect_size	gwas_standard_error	gwas_imputation_status	gwas_n_cases	trait
ENSG00000000457.13	chr1_168894411_A_T_b38	-999856	30	30	0.0223881	0.4447680000000001	0.0509097	0.0665773	Whole_Blood	670	rs114383479	chr1	168894411	T	A	hg38	0.023088023088023088	184305	1.3586797714233398	0.17424808484624	NA	NA	imputed	60801	CARDIoGRAM_C4D_CAD_ADDITIVE


coloc r package was used for preliminary testing. william's abf implementation will follow. example r script to run coloc abf is:

In [ ]:
%%R
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
print(args)

library(dplyr)
library(data.table)
library(coloc)

# example file input_per_gene/Whole_Blood_w_CARDIoGRAM_C4D_CAD_ADDITIVE_for_ENSG00000000457.13_coloc_input_file.txt
# head -2 input_per_gene/Whole_Blood_w_CARDIoGRAM_C4D_CAD_ADDITIVE_for_ENSG00000000457.13_coloc_input_file.txt
# gene_id	eqtl_variant_id	tss_distance	eqtl_ma_samples	eqtl_ma_count	eqtl_maf	eqtl_pval_nominal	eqtl_slope	eqtl_slope_se	tissue	eqtl_sample_size	gwas_variant_id	chromosome	position	eqtl_effect_allele	eqtl_non_effect_allele	current_build	frequency	gwas_sample_size	gwas_zscore	gwas_pvalue	gwas_effect_size	gwas_standard_error	gwas_imputation_status	gwas_n_cases	trait
# ENSG00000000457.13	chr1_168894411_A_T_b38	-999856	30	30	0.0223881	0.4447680000000001	0.0509097	0.0665773	Whole_Blood	670	rs114383479	chr1	168894411	T	A	hg38	0.023088023088023088	184305	1.3586797714233398	0.17424808484624	NA	NA	imputed	60801	CARDIoGRAM_C4D_CAD_ADDITIVE

# assumes input files contains per-gene or per-ldblock variants only

# inputfile = 'Whole_Blood_w_CARDIoGRAM_C4D_CAD_ADDITIVE_for_ENSG00000000457.13_coloc_input_file.txt'
inputfile = args[1]
workdir = '/work/gtex/coloc/'
inputdir = paste(workdir, 'input_per_gene/', sep='')
outputdir = paste(workdir, 'output_per_gene/', sep='')
priorsfile = 'coloc_prior_est_by_enloc_with_traitfile.txt'

tissueid = unlist(strsplit(inputfile, '_w_'))[1]
traitid = unlist(strsplit(inputfile, '_w_'))[2]
geneid = unlist(strsplit(traitid, '_for_'))[2]
geneid = unlist(strsplit(geneid, '_coloc_'))[1]
traitid = unlist(strsplit(traitid, '_for_'))[1]

print(c(tissueid, traitid, geneid))

outfile = paste(outputdir, tissueid, '_w_', traitid, '_for_', geneid, '_coloc_output_file.txt', sep='')
output = c()

inputdata = tbl_df(fread(paste(inputdir, inputfile, sep='')))
inputdata$gwas_varbeta = 1

inputdata = inputdata %>% filter(complete.cases(gwas_zscore)) %>% mutate(max_gwas_zscore = max(abs(gwas_zscore))) 
inputdata = inputdata %>% arrange(desc(abs(gwas_zscore)))

priors = tbl_df(fread(paste(workdir, priorsfile, sep='')))
priorsused = priors %>% filter(trait == traitid & tissue == tissueid)
p1used = priorsused$p1
p2used = priorsused$p2
p12used = priorsused$p12

eqtlinput = list(beta = inputdata$eqtl_slope, varbeta = (inputdata$eqtl_slope_se)**2, N = inputdata$eqtl_sample_size, type = 'quant', snp = inputdata$eqtl_variant_id, MAF = inputdata$eqtl_maf)
gwasinput = list(beta = inputdata$gwas_zscore, varbeta = inputdata$gwas_varbeta, N = inputdata$gwas_sample_size, type = 'quant', snp = inputdata$eqtl_variant_id, MAF = inputdata$frequency)

runcoloc = coloc.abf(eqtlinput, gwasinput, p1=p1used, p2=p2used, p12=p12used)
output = rbind(output, c(unlist(inputdata[1,]), runcoloc$summary))
output = as.data.table(output)
output = mutate(output, PP.H1.abf = ifelse(PP.H0.abf == 1, 0, 'NA'))
output = mutate(output, PP.H2.abf = ifelse(PP.H0.abf == 1, 0, 'NA'))
output = mutate(output, PP.H3.abf = ifelse(PP.H0.abf == 1, 0, 'NA'))
output = mutate(output, PP.H4.abf = ifelse(PP.H0.abf == 1, 0, 'NA'))

# mutate(df, x=ifelse(a==2,x+1,x))

outputdata = tbl_df(as.data.frame(output)) %>% mutate(PP.H4.abf = as.numeric(as.character(PP.H4.abf))) %>% arrange(desc(PP.H4.abf))
write.table(outputdata, file=outfile, quote=F, col.names=T, row.names=F, sep='\t')  		




example output also contains the input file, and resulting abf posterior probabilities PP1, PP2, PP3 and PP4

In [9]:
!cat output_per_gene/Whole_Blood_w_CARDIoGRAM_C4D_CAD_ADDITIVE_for_ENSG00000000457.13_coloc_output_file.txt

gene_id	eqtl_variant_id	tss_distance	eqtl_ma_samples	eqtl_ma_count	eqtl_maf	eqtl_pval_nominal	eqtl_slope	eqtl_slope_se	tissue	eqtl_sample_size	gwas_variant_id	chromosome	position	eqtl_effect_allele	eqtl_non_effect_allele	current_build	frequency	gwas_sample_size	gwas_zscore	gwas_pvalue	gwas_effect_size	gwas_standard_error	gwas_imputation_status	gwas_n_cases	trait	gwas_varbeta	nsnps	PP.H0.abf	PP.H1.abf	PP.H2.abf	PP.H3.abf	PP.H4.abf
ENSG00000000457.13	chr1_168894411_A_T_b38	-999856	30	30	0.0223881	0.444768	0.0509097	0.0665773	Whole_Blood	670	rs114383479	chr1	168894411	T	A	hg38	0.0230880230880231	184305	1.35867977142334	0.17424808484624	NA	NA	imputed	60801	CARDIoGRAM_C4D_CAD_ADDITIVE	1	7210	8.25588247271532e-22	0.245066847408069	2.50289758883083e-21	0.742873226058844	0.0120599265330898
